<a href="https://colab.research.google.com/github/sid5354535457N/Assignmernt/blob/main/Groq_AI_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q groq openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.0 MB/s eta 0:00:00


In [2]:
import os
import json
from openai import OpenAI
from getpass import getpass

In [5]:
try:
    # Attempt to get the key from Colab secrets first
    from google.colab import userdata
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    if GROQ_API_KEY is None:
        raise ImportError
    print("Fetched Groq API key from Colab secrets.")
except (ImportError, KeyError):
    # If secrets don't work, fall back to a secure prompt
    print("Could not find Groq API key in Colab secrets.")
    GROQ_API_KEY = getpass("Please enter your Groq API key: ")


Fetched Groq API key from Colab secrets.


In [6]:
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1",
)

In [25]:
MODEL = 'openai/gpt-oss-20b'

In [26]:
print(f"\n✅ Setup complete. Using model: {MODEL}")


✅ Setup complete. Using model: openai/gpt-oss-20b


In [27]:
print("\n" + "="*60)
print("TASK 1: CONVERSATION HISTORY MANAGEMENT")
print("="*60)


TASK 1: CONVERSATION HISTORY MANAGEMENT


In [28]:
class ConversationManager:
    """
    Manages a conversation history, including truncation and periodic summarization.
    """
    def __init__(self, system_prompt: str, summarize_every_k: int = 3, model: str = MODEL):
        self.system_prompt = {"role": "system", "content": system_prompt}
        self.history = [self.system_prompt]
        self.summarize_every_k = summarize_every_k
        self.model = model
        self.run_count = 0

    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})

    def _summarize_conversation(self, conversation_text: str) -> str:
        print("\n⏳ Summarizing conversation...")
        try:
            summary_prompt = f"Concisely summarize the key points of the following conversation in 1-2 sentences: \n\n{conversation_text}"
            response = client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": summary_prompt}],
                temperature=0.3,
            )
            summary = response.choices[0].message.content
            print("✅ Summarization complete.")
            return summary
        except Exception as e:
            print(f"Error during summarization: {e}")
            return "Summary could not be generated."

    def process_and_get_response(self, user_input: str) -> str:
        self.add_message("user", user_input)
        self.run_count += 1

        if self.run_count % self.summarize_every_k == 0 and len(self.history) > 3:
            print(f"\n--- Triggering summarization (Run #{self.run_count}) ---")
            text_to_summarize = "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.history[:-1]])
            summary = self._summarize_conversation(text_to_summarize)
            latest_user_message = self.history[-1]
            self.history = [
                self.system_prompt,
                {"role": "system", "content": f"Conversation summary so far: {summary}"},
                latest_user_message
            ]
            print("📜 History has been replaced with a summary.")

        try:
            chat_completion = client.chat.completions.create(messages=self.history, model=self.model)
            assistant_response = chat_completion.choices[0].message.content
            self.add_message("assistant", assistant_response)
            return assistant_response
        except Exception as e:
            return f"An error occurred: {e}"

    def display_history(self):
        print("\n--- Current Conversation History ---")
        for message in self.history:
            print(f"[{message['role'].capitalize()}]: {message['content']}")
        print("------------------------------------\n")

    def truncate_history(self, max_turns: int = None, max_chars: int = None):
        if max_turns:
            print(f"✂️ Truncating history to the last {max_turns} turns...")
            num_messages_to_keep = max_turns * 2
            self.history = [self.system_prompt] + self.history[-num_messages_to_keep:]

        if max_chars:
            print(f"✂️ Truncating history to ~{max_chars} characters...")
            current_chars = 0
            new_history = []
            for message in reversed(self.history[1:]):
                current_chars += len(message['content'])
                if current_chars > max_chars:
                    break
                new_history.insert(0, message)
            self.history = [self.system_prompt] + new_history

In [29]:
print("\n### 2.1. DEMONSTRATION: PERIODIC SUMMARIZATION ###")
convo_manager = ConversationManager(system_prompt="You are a helpful travel planner.", summarize_every_k=3)


### 2.1. DEMONSTRATION: PERIODIC SUMMARIZATION ###


In [30]:
print("--- Run 1 ---")
response = convo_manager.process_and_get_response("Hi, I want to plan a trip to Italy.")
print(f"[Assistant]: {response}")

--- Run 1 ---
[Assistant]: 👋 Hello! I’d love to help you design an unforgettable Italian adventure.  
Let’s start by getting a feel for what you’re looking for, and then I’ll sketch out a few sample itineraries (plus tips, budget ideas, and a handy packing list).

---

## 1. Quick “Get‑to‑Know” Questions  
(Feel free to skip any you’re not sure about; I’ll use the defaults.)

| Question | What it helps us decide |
|----------|------------------------|
| **Dates / Season** | Winter (ski, fewer crowds), Spring (blooming countryside), Summer (beaches, festivals), Autumn (wine harvest) |
| **Duration** | 7, 10, 14 days, or a “quick getaway” (3–5) |
| **Budget** | Economy, mid‑range, or luxury (per day: €80, €150, €300+) |
| **Travel Style** | Backpacker, family, honeymoon, solo, culinary, art/history, outdoor adventure |
| **Must‑See** | Rome, Florence, Venice, Amalfi Coast, Tuscany, Sicily, etc. |
| **Cuisine Focus** | Pizza, pasta, seafood, gelato, regional specialties, food tours |
| **

In [31]:
print("\n--- Run 2 ---")
response = convo_manager.process_and_get_response("I'm interested in historical sites and good food. I'm thinking of visiting in the spring.")
print(f"[Assistant]: {response}")


--- Run 2 ---
[Assistant]: Great! 🌟 Spring is the perfect time for Italy—mild weather, blooming landscapes, and a vibrant culinary season.  
Below is a **sample 12‑day itinerary** that balances *historical wonders* with *food‑centric experiences*.  
I’ll also give you a rough budget, key transportation tips, and some “must‑try” dishes for each stop.  

---

## 🎯 12‑Day “History & Food” Italy Itinerary (Spring)

| Day | City | Highlights (History) | Food Focus | Suggested Lunch/Evening |
|-----|------|----------------------|------------|------------------------|
| **1** | **Rome** | Colosseum, Roman Forum, Palatine Hill | Italian classics | *Pasta carbonara* at a trattoria |
| **2** | Rome | Vatican Museums, Sistine Chapel, St. Peter’s | Espresso, gelato | *Pizzette* at Pizzarium |
| **3** | Rome → **Florence** | Duomo, Uffizi, Palazzo Vecchio | Tuscan cheese & wine | *Bistecca alla fiorentina* |
| **4** | Florence | Boboli Gardens, Piazzale Michelangelo | Truffle & porcini soup | *Gel

In [32]:
print("\n--- Run 3 (Summarization will trigger here) ---")
response = convo_manager.process_and_get_response("What are the top 3 cities I should visit for that?")
print(f"[Assistant]: {response}")
convo_manager.display_history()


--- Run 3 (Summarization will trigger here) ---

--- Triggering summarization (Run #3) ---

⏳ Summarizing conversation...
✅ Summarization complete.
📜 History has been replaced with a summary.
[Assistant]: Here are the three Italian cities that give you the richest blend of history and culinary delight for a spring getaway:

| # | City | Why it’s a must‑visit | Highlighted historic sites | Signature foods |
|---|------|-----------------------|----------------------------|-----------------|
| 1 | **Rome** | The Eternal City is a living museum, and spring is when the crowds are lighter and the temperatures are comfortable. | Colosseum, Roman Forum, Pantheon, Trastevere’s medieval streets | Carbonara, Supplì, Gelato (Baci) |
| 2 | **Florence** | The cradle of the Renaissance, it’s a compact city where art and food are inseparable. | Duomo (Cathedral), Uffizi, Ponte Vecchio, Boboli Gardens | Bistecca alla Fiorentina, Ribollita, Florentine steak |
| 3 | **Venice** | Unique architecture, can

In [33]:
print("\n### 2.2. DEMONSTRATION: TRUNCATION ###")
convo_manager.truncate_history(max_turns=2)
convo_manager.display_history()

convo_manager.truncate_history(max_chars=200)
convo_manager.display_history()


### 2.2. DEMONSTRATION: TRUNCATION ###
✂️ Truncating history to the last 2 turns...

--- Current Conversation History ---
[System]: You are a helpful travel planner.
[System]: You are a helpful travel planner.
[System]: Conversation summary so far: The user wants a spring trip to Italy focused on historical sites and good food, and the assistant responded with a detailed 12‑day itinerary that balances major landmarks (e.g., Rome’s Colosseum, Florence’s Duomo, Pompeii) with culinary highlights, along with budget estimates, transport tips, and accommodation suggestions.
[User]: What are the top 3 cities I should visit for that?
[Assistant]: Here are the three Italian cities that give you the richest blend of history and culinary delight for a spring getaway:

| # | City | Why it’s a must‑visit | Highlighted historic sites | Signature foods |
|---|------|-----------------------|----------------------------|-----------------|
| 1 | **Rome** | The Eternal City is a living museum, and sprin

In [34]:
print("\n" + "="*60)
print("TASK 2: JSON SCHEMA CLASSIFICATION & EXTRACTION")
print("="*60)


TASK 2: JSON SCHEMA CLASSIFICATION & EXTRACTION


In [35]:
USER_DETAILS_SCHEMA = {
    "type": "function",
    "function": {
        "name": "extract_user_info",
        "description": "Extracts user information from a given text.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The full name of the user."},
                "email": {"type": "string", "description": "The email address of the user."},
                "phone": {"type": "string", "description": "The phone number of the user."},
                "location": {"type": "string", "description": "The city and country, e.g., 'San Francisco, USA'."},
                "age": {"type": "integer", "description": "The age of the user."}
            },
            "required": []
        }
    }
}

In [36]:
def extract_details(text_input: str, model: str = MODEL):
    print(f"\n--- Parsing Chat: \"{text_input}\" ---")
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": text_input}],
            tools=[USER_DETAILS_SCHEMA],
            tool_choice="auto"
        )
        tool_calls = response.choices[0].message.tool_calls
        if tool_calls:
            function_args = json.loads(tool_calls[0].function.arguments)
            print("✅ Extraction successful.")
            return function_args
        else:
            print("ℹ️ Model did not find any details to extract.")
            return {}
    except Exception as e:
        print(f"An error occurred during extraction: {e}")
        return {"error": str(e)}

In [37]:
print("\n### 3.3. DEMONSTRATION: EXTRACTING FROM SAMPLE CHATS ###")
sample_chats = [
    "Hi, my name is Jane Doe and I'm 28. My email is jane.d@email.com, and I live in New York, USA. My phone is 123-456-7890.",
    "I'm John Smith from London. You can reach me at john.s@web.co.uk.",
    "Can you please tell me what the weather is like today?"
]


### 3.3. DEMONSTRATION: EXTRACTING FROM SAMPLE CHATS ###


In [38]:
for chat in sample_chats:
    extracted_data = extract_details(chat)
    print("--- Extracted Information ---")
    print(json.dumps(extracted_data, indent=2))
    print("-" * 30)


--- Parsing Chat: "Hi, my name is Jane Doe and I'm 28. My email is jane.d@email.com, and I live in New York, USA. My phone is 123-456-7890." ---
✅ Extraction successful.
--- Extracted Information ---
{
  "age": 28,
  "email": "jane.d@email.com",
  "location": "New York, USA",
  "name": "Jane Doe",
  "phone": "123-456-7890"
}
------------------------------

--- Parsing Chat: "I'm John Smith from London. You can reach me at john.s@web.co.uk." ---
✅ Extraction successful.
--- Extracted Information ---
{
  "email": "john.s@web.co.uk",
  "location": "London",
  "name": "John Smith"
}
------------------------------

--- Parsing Chat: "Can you please tell me what the weather is like today?" ---
ℹ️ Model did not find any details to extract.
--- Extracted Information ---
{}
------------------------------
